In [1]:
import pandas as pd
from datetime import datetime, timedelta
from api import static
from setup import setup
import sqlalchemy as sa


def get_last_status(engine):
    query = sa.text("select * from elements_summary;")
    with engine.begin() as conn:
        elements = pd.read_sql(query, con=conn)
    return elements


def get_live_status():
    live_status = static()
    elements = live_status.get_elements_summary()
    return elements


if __name__ == '__main__':
    today = datetime.now()
    engine_changes = setup()
    engine_live = setup()
    last_all = get_last_status(engine_changes)
    engine_changes.dispose()
    live_all = get_live_status()
    live = live_all.loc[live_all['cost_change_event'] != 0,['id', 'now_cost']]
    last = last_all.loc[last_all['cost_change_event'] != 0,['id', 'now_cost']]

    # GET CHANGES LIST
    last_list = ['-'.join([str(i) for i in e])
                 for e in last.to_numpy()]
    live_list = ['-'.join([str(i) for i in e])
                 for e in live.to_numpy()]
    diff = list(set(live_list).difference(set(last_list)))

    # IF CHANGES AVAILABLE
    # if len(diff) > 0:
    changes_list = [int(e.split('-')[0]) for e in diff]

    # GET DIFFERENCE
    last = last_all.loc[last_all['id'].isin(changes_list), ['id', 'now_cost']]
    live = live.loc[live['id'].isin(changes_list), ['id', 'now_cost']]
    changes = last.join(live.set_index('id'), on='id',
                        how='right', rsuffix='_live', lsuffix='_last')
    changes['cost_difference'] = changes['now_cost_live'] - changes['now_cost_last']
    changes['date_change'] = today.date().isoformat()

    # PUSH TO DB
    changes.to_sql(name='test1', con=engine_changes, if_exists='replace', index=False)
    engine_changes.dispose()
        # live_all.to_sql(name='test2', con=engine_live, if_exists='replace', index=False)

Error closing cursor
Traceback (most recent call last):
  File "c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sqlalchemy\engine\base.py", line 2203, in _safe_close_cursor
    cursor.close()
  File "c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\mysql\connector\cursor_cext.py", line 495, in close
    self._cnx.handle_unread_result()
  File "c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\mysql\connector\connection_cext.py", line 932, in handle_unread_result
    raise InternalError("Unread result found")
mysql.connector.errors.InternalError: Unread result found
Error closing cursor
Traceback (most recent call last):
  File "c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sqlalchemy\engine\base.py", line 2203, in _safe_close_cursor
    cursor.close()
  File "c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\mysql\connector\cursor_cext.py", line 495, in close
    self._c

In [2]:
changes

,id,now_cost_last,now_cost_live,cost_difference,date_change
118,104,45.0,44.0,-1.0,2023-11-26
162,143,66.0,65.0,-1.0,2023-11-26
250,216,69.0,70.0,1.0,2023-11-26
253,362,52.0,53.0,1.0,2023-11-26
280,228,46.0,47.0,1.0,2023-11-26
304,245,40.0,41.0,1.0,2023-11-26
508,617,71.0,70.0,-1.0,2023-11-26
547,433,79.0,78.0,-1.0,2023-11-26
558,186,44.0,43.0,-1.0,2023-11-26
721,557,55.0,56.0,1.0,2023-11-26
